In [287]:
import cirq
import numpy as np
from sklearn.preprocessing import normalize
import sympy as sp
from cirq.contrib.svg import SVGCircuit
import tensorflow_quantum as tfq
from tensorflow_quantum.python import util
import re
import collections

In [281]:
def beta(s, j, x):
    index_num = (2*j-1)*(2**(s-1))
    index_den = (j-1)*(2**s)
    
    num = np.sqrt(np.sum(abs(x[index_num : index_num+2**(s-1)])**2))
    den = np.sqrt(np.sum(abs(x[index_den : index_den+2**(s)])**2))
    
    if den == 0:
        beta = 0
    else:
        beta = 2*np.arcsin(num/den)
    return beta

In [282]:
def locate_x(curr_j, prev_j, length):
    curr_bin = bin(curr_j)[2:].zfill(length)
    prev_bin = bin(prev_j)[2:].zfill(length)
    return [i for i, (x,y) in enumerate(zip(curr_bin,prev_bin)) if x!=y]

In [283]:
def amplitude_embedding(x):
    n = int(np.log2(len(x)))
    qubits = cirq.GridQubit.rect(1, n)
    circuit = cirq.Circuit()
    
    circuit += cirq.ry(beta(n, 1, x))(qubits[0])
    
    for i in range(1,n):
        # We can have at most i control bits
        # Total possibilities is therefore 2^i
        controls = 2**i
        
        control_qubits = [qubits[c] for c in range(i+1)]
        circuit += cirq.ControlledGate(sub_gate=cirq.ry(beta(n-i, controls, x)), 
                                       num_controls=len(control_qubits)-1)(*control_qubits)
        
        for j in range(1, controls):
            for loc in locate_x(controls-j-1, controls-j, i):
                circuit += cirq.X(qubits[loc])
                
            circuit += cirq.ControlledGate(sub_gate=cirq.ry(beta(n-i, controls-j, x)), 
                                       num_controls=len(control_qubits)-1)(*control_qubits)
            
        for k in range(i):
            circuit += cirq.X(qubits[k])

    return circuit

In [284]:
x = np.arange(1,5,1)
d = np.sqrt(np.sum(np.square(x)))
x = x/d
print('amplitudes:',x)
print('probs:',np.square(x))
qubits = cirq.GridQubit.rect(1,int(np.ceil(np.log2(len(x)))))
qc = cirq.Circuit()
qc += amplitude_embedding(x)
qc += cirq.measure(*qubits, key='result')

amplitudes: [0.18257419 0.36514837 0.54772256 0.73029674]
probs: [0.03333333 0.13333333 0.3        0.53333333]


In [285]:
s=cirq.Simulator()
shots = 10000
samples=s.run(qc, repetitions=shots)
res = dict(samples.histogram(key="result"))
for key, value in res.items():
    res[key] = value/shots
res

{3: 0.5311, 2: 0.3031, 1: 0.1318, 0: 0.034}

In [288]:
od = collections.OrderedDict(sorted(res.items()))

for k, v in od.items():
    print(k,v)

0 0.034
1 0.1318
2 0.3031
3 0.5311


In [14]:
x = np.arange(5,9,1)
d = np.sqrt(np.sum(np.square(x)))
x = x/d
print('amplitudes:',x)
print('probs:',np.square(x))
qubits = cirq.GridQubit.rect(1,int(np.ceil(np.log2(len(x)))))
qc = cirq.Circuit()
qc += amplitude_embedding(x)
qc += cirq.measure(*qubits, key='result')

amplitudes: [0.37904902 0.45485883 0.53066863 0.60647843]
probs: [0.14367816 0.20689655 0.2816092  0.36781609]


In [15]:
qc

(0, 0): ───Ry(0.597π)───@────────────X───@────────────X───M('result')───
                        │                │                │
(0, 1): ────────────────Ry(0.542π)───────Ry(0.558π)───────M─────────────

In [16]:
s=cirq.Simulator()
shots = 10000
samples=s.run(qc, repetitions=shots)
res = dict(samples.histogram(key="result"))
for key, value in res.items():
    res[key] = value/shots
res

{3: 0.3677, 1: 0.2071, 0: 0.1455, 2: 0.2797}

## Amplitude Encoding using module

In [17]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

from importlib.util import find_spec
if find_spec("qml_hep_lhc") is None:
    import sys
    sys.path.append('..')

In [110]:
from qml_hep_lhc.encodings import AmplitudeMap
from qml_hep_lhc.data import ElectronPhoton
import sympy as sp
import argparse
from qml_hep_lhc.layers.utils import get_count_of_qubits, get_num_in_symbols
from qml_hep_lhc.layers.utils import symbols_in_expr_map, resolve_formulas
from tensorflow import pad, constant
import tensorflow as tf

In [294]:
x = np.array([1,2,3,4], dtype=np.float32)

In [254]:
n_qubits = get_count_of_qubits('AmplitudeMap',4)
n_inputs = get_num_in_symbols('AmplitudeMap', 4)

n_qubits, n_inputs

(2, 4)

In [255]:
in_symbols = sp.symbols(f'x0:{n_inputs}')
in_symbols = np.asarray(in_symbols).reshape(1,n_inputs)

in_symbols

array([[x0, x1, x2, x3]], dtype=object)

In [295]:
x = x.reshape(1,-1)
x.shape

(1, 4)

In [257]:
np.sqrt(np.sum(x**2))

5.477226

In [258]:
qubits = cirq.GridQubit.rect(1,n_qubits)
circuit = cirq.Circuit()
circuit += AmplitudeMap().build(qubits,in_symbols[0])

In [259]:
circuit

(0, 0): ───Ry(2*pi*asin((Abs(x2)**2 + Abs(x3)**2)**0.5*(Abs(x0)**2 + Abs(x1)**2 + Abs(x2)**2 + Abs(x3)**2)**(-0.5)))───@───────────────────────────────────────────────────────────────X───@───────────────────────────────────────────────────────────────X───
                                                                                                                       │                                                                   │
(0, 1): ───────────────────────────────────────────────────────────────────────────────────────────────────────────────Ry(2*pi*asin((Abs(x2)**2 + Abs(x3)**2)**(-0.5)*Abs(x3)**1.0))───────Ry(2*pi*asin((Abs(x0)**2 + Abs(x1)**2)**(-0.5)*Abs(x1)**1.0))───────

In [260]:
circuit, expr_map = cirq.flatten(circuit)
raw_in_symbols = symbols_in_expr_map(expr_map)
data_expr = list(expr_map)

In [261]:
raw_in_symbols

[x0, x1, x2, x3]

In [262]:
data_expr

[2*asin((Abs(x2)**2 + Abs(x3)**2)**0.5*(Abs(x0)**2 + Abs(x1)**2 + Abs(x2)**2 + Abs(x3)**2)**(-0.5)),
 2*asin((Abs(x2)**2 + Abs(x3)**2)**(-0.5)*Abs(x3)**1.0),
 2*asin((Abs(x0)**2 + Abs(x1)**2)**(-0.5)*Abs(x1)**1.0)]

In [263]:
circuit

(0, 0): ───Ry(pi*<2*asin((Abs(x2)**2 + Abs(x3)**2)**0.5*(Abs(x0)**2 + Abs(x1)**2 + Abs(x2)**2 + Abs(x3)**2)**(-0.5))>)───@─────────────────────────────────────────────────────────────────X───@─────────────────────────────────────────────────────────────────X───
                                                                                                                         │                                                                     │
(0, 1): ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────Ry(pi*<2*asin((Abs(x2)**2 + Abs(x3)**2)**(-0.5)*Abs(x3)**1.0)>)───────Ry(pi*<2*asin((Abs(x0)**2 + Abs(x1)**2)**(-0.5)*Abs(x1)**1.0)>)───────

In [264]:
list(expr_map.values())

[<2*asin((Abs(x2)**2 + Abs(x3)**2)**0.5*(Abs(x0)**2 + Abs(x1)**2 + Abs(x2)**2 + Abs(x3)**2)**(-0.5))>,
 <2*asin((Abs(x2)**2 + Abs(x3)**2)**(-0.5)*Abs(x3)**1.0)>,
 <2*asin((Abs(x0)**2 + Abs(x1)**2)**(-0.5)*Abs(x1)**1.0)>]

In [265]:
input_resolver = resolve_formulas(data_expr, raw_in_symbols)

In [297]:
padding = n_inputs - x.shape[1]
if padding:
    x = pad(x, constant([ [0,0], [0, padding]]), constant_values=1.0)
x, _ = tf.linalg.normalize(x, axis = 1)

In [298]:
np.sqrt(np.sum(x**2))

0.99999994

In [299]:
x.numpy()

array([[0.18257418, 0.36514837, 0.5477225 , 0.73029673]], dtype=float32)

In [270]:
resolved_x = input_resolver(x)

In [272]:
resolved_x.numpy()

array([[2.3005242, 1.8545904, 2.2142973]], dtype=float32)

In [273]:
resolved_x.shape

TensorShape([1, 3])

In [274]:
resolver = cirq.ParamResolver()

In [276]:
qc = cirq.resolve_parameters(circuit, { j: resolved_x[0][i].numpy()  for i,j  in enumerate(list(expr_map.values()))})
qc

(0, 0): ───Ry(-1.7π)───@───────────X───@────────────X───
                       │               │
(0, 1): ───────────────Ry(1.85π)───────Ry(-1.79π)───────

In [277]:
qc += cirq.measure(*qubits, key='result')

In [290]:
s=cirq.Simulator()
shots = 10000
samples=s.run(qc, repetitions=shots)
res = dict(samples.histogram(key="result"))
for key, value in res.items():
    res[key] = value/shots
res

{2: 0.2874, 3: 0.5453, 1: 0.1325, 0: 0.0348}

In [291]:
od = collections.OrderedDict(sorted(res.items()))

for k, v in od.items():
    print(k,v)

0 0.0348
1 0.1325
2 0.2874
3 0.5453


In [300]:
probs = np.square(x)[0]
for i, j in enumerate(probs):
    print(i,j)

0 0.03333333
1 0.13333333
2 0.29999995
3 0.5333333


In [303]:
list(expr_map.keys())

[2*asin((Abs(x2)**2 + Abs(x3)**2)**0.5*(Abs(x0)**2 + Abs(x1)**2 + Abs(x2)**2 + Abs(x3)**2)**(-0.5)),
 2*asin((Abs(x2)**2 + Abs(x3)**2)**(-0.5)*Abs(x3)**1.0),
 2*asin((Abs(x0)**2 + Abs(x1)**2)**(-0.5)*Abs(x1)**1.0)]

In [305]:
list(expr_map)

[2*asin((Abs(x2)**2 + Abs(x3)**2)**0.5*(Abs(x0)**2 + Abs(x1)**2 + Abs(x2)**2 + Abs(x3)**2)**(-0.5)),
 2*asin((Abs(x2)**2 + Abs(x3)**2)**(-0.5)*Abs(x3)**1.0),
 2*asin((Abs(x0)**2 + Abs(x1)**2)**(-0.5)*Abs(x1)**1.0)]

In [306]:
list(expr_map.values())

[<2*asin((Abs(x2)**2 + Abs(x3)**2)**0.5*(Abs(x0)**2 + Abs(x1)**2 + Abs(x2)**2 + Abs(x3)**2)**(-0.5))>,
 <2*asin((Abs(x2)**2 + Abs(x3)**2)**(-0.5)*Abs(x3)**1.0)>,
 <2*asin((Abs(x0)**2 + Abs(x1)**2)**(-0.5)*Abs(x1)**1.0)>]